### ETL, Postgres DB
This is a test notebook for ETL on postgres SQL, a dumped SQL from after Kafka. 

IP: 192.168.7.114, hua's test VM, CentOS. Please make sure the docker_db_1 container is running and some pcap files has been processed manually or automatically.

192.168.7.12: the offical distributed VM clusters, db running on it.

192.168.7.11:5601: Kibana...

2018 May 29, fix type issue on ETL container, re-run on the sample cap(03-22-13.pcap, 116 MB) on distributed system.

In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import sys

/Users/ustc-weihua/anaconda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# https://sshtunnel.readthedocs.io/en/latest/
def test_conn(hostip = "192.168.7.114"):
    PORT=5432
    with SSHTunnelForwarder(
        hostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        print("Connected!\n")

        cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
        print(cursor.fetchall())

test_conn("192.168.7.12")

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
Connected!

[('dns',), ('conn',), ('files',), ('http',)]


In [3]:
import numpy as np
import pandas as pd

In [4]:
# function to print the column name for the table
def db_print_colnames(dbhostip, tablename):
    PORT=5432
    with SSHTunnelForwarder(
        dbhostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        #print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        
        # execute our Query
        sqlquery = "select column_name from information_schema.columns where table_name='" + tablename + "'"
        cursor.execute(sqlquery)
        # retrieve the records from the database
        colnames = cursor.fetchall()
        return colnames

for tablename in ['conn', 'files', 'http', 'dns']:
    print(tablename)
    print( [x[0] for x in db_print_colnames("192.168.7.12", tablename)] )

conn
['duration', 'resp_cc', 'ts', 'uid', 'history', 'orig_cc', 'conn_state', 'orig_bytes', 'resp_bytes', 'service', 'kafka_offset']
files
['tx_hosts', 'filename', 'ts', 'fuid', 'source', 'extracted', 'mime_type', 'rx_hosts', 'conn_uids', 'kafka_offset']
http
['uid', 'status_code', 'info_code', 'id.orig_p', 'username', 'response_body_len', 'id.resp_p', 'host', 'ts', 'info_msg', 'request_body_len', 'id.resp_h', 'method', 'user_agent', 'referrer', 'resp_mime_types', 'uri', 'filename', 'status_msg', 'id.orig_h', 'orig_mime_types', 'kafka_offset']
dns
['RD', 'rejected', 'qtype_name', 'TTL', 'qtype', 'query', 'Z', 'answers', 'qclass', 'RA', 'rcode', 'TC', 'AA', 'uid', 'qclass_name', 'ts', 'rcode_name', 'kafka_offset']


In [5]:
# function to convert the table in the postgres database to pandas dataframe
def db_to_df(dbhostip, tablename):
    PORT=5432
    with SSHTunnelForwarder(
        dbhostip,
        ssh_username="root",
        ssh_password="Helios12$",
        remote_bind_address=('localhost', PORT),
        local_bind_address=('localhost', PORT)
    ):
        #Define our connection string
        conn_string = "host='localhost' dbname='helios' user='helios' password='helios'"
 
        # print the connection string we will use to connect
        print("Connecting to database\n ->%s" % (conn_string))
 
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
 
        # conn.cursor will return a cursor object, you can use this cursor to perform queries
        cursor = conn.cursor()
        
        # get some records
        # execute our Query
        #sqlquery = "select * from " + tablename + " limit 10"
        sqlquery = "select * from " + tablename
        cursor.execute(sqlquery)
        # retrieve the records from the database
        records = cursor.fetchall()
        cursor.execute("select column_name from information_schema.columns where table_name='dns'")
        #print (cursor.fetchall())
        
        # get the column names
        # execute our Query
        sqlquery = "select column_name from information_schema.columns where table_name='" + tablename + "'"
        cursor.execute(sqlquery)
        # retrieve the records from the database
        colnames = [x[0] for x in cursor.fetchall()]
        
        df = pd.DataFrame(records, columns=colnames)
        return df
        
#mytest_df = db_to_df("192.168.7.114", "files")
#mytest_df.head()

## table investigation

First, all possible logs are in https://www.bro.org/sphinx/script-reference/log-files.html

Several types: Network Protocols(conn.log, http.log, dns.log...), files(files.log), NetControl, Detection, Network Observations, Miscellaneous, Bro Diagnostics.

Note, all tables have "ts" column, and this ts is from bro package, which means the real time stamp substract 1970-01-01 00:00:00. More information: https://www.bro.org/sphinx-git/scripts/policy/protocols/ssl/validate-sct.bro.html

In general, what is kafka_offset?

### table investigation, conn

IP, TCP, UDP and ICMP connection details.

column name | description
----------- | -----------
service | dynamically detected protocol, if any
duration | time of the last packet seen - time of the first packet seen
resp_bytes | Responder payload bytes; from sequence numbers if TCP
orig_bytes | Originator payload bytes; from sequence numbers if TCP
history | Connection state history (see conn.log:history table)?
ts | tims stamp from bro
resp_cc | RESP GeoIP Country Code
uid | Unique ID of Connection
orig_cc | ORIG GeoIP Country Code
conn_state | Connection state (see conn.log:conn_state table)?
kafka_offset | ?

- How about connection state and connection state history?

In [6]:
#conn_df = db_to_df("192.168.7.114", "conn")
conn_df = db_to_df("192.168.7.12", "conn")
print(len(conn_df))
conn_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
7467


,duration,resp_cc,ts,uid,history,orig_cc,conn_state,orig_bytes,resp_bytes,service,kafka_offset
0,135.604092,,1.527292e+09,CQz9KJfYyx8zhvb4c,,,OTH,160.0,0.0,,1339
1,135.289071,,1.527292e+09,C7g80v1Ktn3vtizODl,,,OTH,64.0,0.0,,1340
2,135.289071,,1.527292e+09,C7g80v1Ktn3vtizODl,,,OTH,64.0,0.0,,1340
3,136.125039,,1.527292e+09,CtGyvg21JXVA2kkPdg,,,OTH,160.0,0.0,,1341
4,136.125039,,1.527292e+09,CtGyvg21JXVA2kkPdg,,,OTH,160.0,0.0,,1341


In [7]:
conn_df.dtypes

duration        float64
resp_cc          object
ts              float64
uid              object
history          object
orig_cc          object
conn_state       object
orig_bytes      float64
resp_bytes      float64
service          object
kafka_offset      int64
dtype: object

### table investigation, files

File analysis results

column name | description
----------- | -----------
fuid      | identifier for a single file
filename  | If available, filename from source; frequently the “Content-Disposition” headers in network protocols
tx_hosts  | if transferred via network, host(s) that sourced the data
conn_uids | Connection UID(s) over which the file was transferred
ts        | time stamp from bro
source    | An identification of the source of the file data
rx_hosts  | if transferred via network, host(s) that received the data
mime_type | Libmagic sniffed file type?
extracted | Local filename of extracted files, if enabled
kafka_offset | ?

- what is mime_type?

In [8]:
#files_df = db_to_df("192.168.7.114", "files")
files_df = db_to_df("192.168.7.12", "files")
print(len(files_df))
files_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
228


,tx_hosts,filename,ts,fuid,source,extracted,mime_type,rx_hosts,conn_uids,kafka_offset
0,{192.168.3.111},,1.521700e+09,FB52dEaGdh7RRUbu7,HTTP,,,{192.168.3.61},{CVKVtNYd64Rn8AzSf},114
1,{192.168.3.111},,1.521700e+09,FB52dEaGdh7RRUbu7,HTTP,,,{192.168.3.61},{CVKVtNYd64Rn8AzSf},114
2,{192.168.3.111},,1.521701e+09,FOS6qT17ymqq7JNUI6,HTTP,,,{192.168.3.121},{CQU8TJ1fdgtyIKDKMc},115
3,{192.168.3.111},,1.521701e+09,FOS6qT17ymqq7JNUI6,HTTP,,,{192.168.3.121},{CQU8TJ1fdgtyIKDKMc},115
4,{172.16.1.29},,1.521701e+09,FGIt8E4nXBFXCrB16b,HTTP,HTTP-FGIt8E4nXBFXCrB16b.html,text/html,{192.168.3.183},{C3mM0q1V26IA9IhQUl},116


In [9]:
files_df.dtypes

tx_hosts         object
filename         object
ts              float64
fuid             object
source           object
extracted        object
mime_type        object
rx_hosts         object
conn_uids        object
kafka_offset      int64
dtype: object

### table investigation, http

column name | description
----------- | -----------
id.orig_p   | 
method      | GET, POST
response_body_len | the length of the packet
uid | hashed user id, to pretect the privacy
orig_mime_types | ?
info_code  | ?
referrer   | ?
filename   | the name of downloaded file?
user_agent | value of user agent header
info_msg   | ?
ts         | tims stamp from bro
resp_mime_types | ?
id.resp_h | ?
host | ?
id.resp_p | ?
username  | ?
status_msg |
request_body_len | ?
id.orig_h | ?
uri | ?
status_code | ?
kafka_offset | ?

In [10]:
#http_df = db_to_df("192.168.7.114", "http")
http_df = db_to_df("192.168.7.12", "http")
print(len(http_df))
http_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
162


,uid,status_code,info_code,id.orig_p,username,response_body_len,id.resp_p,host,ts,info_msg,...,method,user_agent,referrer,resp_mime_types,uri,filename,status_msg,id.orig_h,orig_mime_types,kafka_offset
0,CVKVtNYd64Rn8AzSf,200.0,None,62321,,4,8080,Officescan,1.521700e+09,,...,GET,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A06AF...,,OK,192.168.3.61,,81
1,CVKVtNYd64Rn8AzSf,200.0,None,62321,,4,8080,Officescan,1.521700e+09,,...,GET,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A06AF...,,OK,192.168.3.61,,81
2,CQU8TJ1fdgtyIKDKMc,200.0,None,53051,,4,8080,Officescan,1.521701e+09,,...,GET,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A07AF...,,OK,192.168.3.121,,82
3,CQU8TJ1fdgtyIKDKMc,200.0,None,53051,,4,8080,Officescan,1.521701e+09,,...,GET,OSCE;12.0;1775;dd9c3434-f845-4bd8-86f7-cde1f48...,,,/ss/q/?BF=1748000400&LCRC=AB795571F586FB4A07AF...,,OK,192.168.3.121,,82
4,C3mM0q1V26IA9IhQUl,407.0,None,38143,,2147,3128,detik.com,1.521701e+09,,...,GET,Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko...,,{text/html},http://detik.com/,,Proxy Authentication Required,192.168.3.183,,83


In [11]:
http_df.dtypes

uid                   object
status_code          float64
info_code             object
id.orig_p              int64
username              object
response_body_len      int64
id.resp_p              int64
host                  object
ts                   float64
info_msg              object
request_body_len       int64
id.resp_h             object
method                object
user_agent            object
referrer              object
resp_mime_types       object
uri                   object
filename              object
status_msg            object
id.orig_h             object
orig_mime_types       object
kafka_offset           int64
dtype: object

### table investigation, dns

DNS query/response details

column name | description
----------- | -----------
qclass | Value specifying the query class?
qtype | Value specifying the query type?
AA | Authoritative Answer. T = server is authoritative for query?
rcode | 
RA | 
query | 
TTL | 
ts | time stamp from bro
TC | 
RD | 
answers | 
qtype_name | 
uid | 
rcode_name | 
rejected | 
Z | 
qclass_name | 
kafka_offset | ?

- em... basically need more understanding...

In [12]:
#dns_df = db_to_df("192.168.7.114", "dns")
dns_df = db_to_df("192.168.7.12", "dns")
print(len(dns_df))
dns_df.head()

Connecting to database
 ->host='localhost' dbname='helios' user='helios' password='helios'
1292


,RD,rejected,qtype_name,TTL,qtype,query,Z,answers,qclass,RA,rcode,TC,AA,uid,qclass_name,ts,rcode_name,kafka_offset
0,True,False,A,None,1.0,mirrorlist.centos.org,0,"{212.69.166.138,216.176.179.218,67.219.148.138...",1.0,True,0.0,False,False,CJXrTQ2qZCJzvYDxj1,C_INTERNET,1.521699e+09,NOERROR,646
1,True,False,A,None,1.0,mirrorlist.centos.org,0,None,1.0,False,NaN,False,False,CMdtmkOEqb83HGda4,C_INTERNET,1.521699e+09,None,647
2,True,False,A,None,1.0,mirror.centos.org,0,None,1.0,False,NaN,False,False,CxJSHkUwCrLreN3Li,C_INTERNET,1.521699e+09,None,648
3,True,True,A,None,1.0,corefouat1,0,None,1.0,False,2.0,False,False,CoZAMX2dj9qIALsNvc,C_INTERNET,1.521699e+09,SERVFAIL,649
4,True,True,AAAA,None,28.0,corefouat1,0,None,1.0,False,2.0,False,False,CoZAMX2dj9qIALsNvc,C_INTERNET,1.521699e+09,SERVFAIL,650


In [13]:
dns_df.dtypes

RD                 bool
rejected           bool
qtype_name       object
TTL              object
qtype           float64
query            object
Z                 int64
answers          object
qclass          float64
RA                 bool
rcode           float64
TC                 bool
AA                 bool
uid              object
qclass_name      object
ts              float64
rcode_name       object
kafka_offset      int64
dtype: object

### The end of this notebook